# Projekt 1 - Hőmérséklet kiolvasása

Első projektünkben megismerkedünk a DS18b20 szenzorral és, hogy hogyan tudunk belőle pythonból információt kinyerni. Ennek első feltétele, hogy a bevezetőben leírt 1-Wire protokolt már aktiváltuk a Raspberry Pi-on. 

A hőmérséklet kiolvasásának a logikája a következő lesz. Pythonból lefuttatjuk a terminál parancsokat, amik lehetővé teszik a számítógép és a thermisztor közti kommunikációt, majd szoftveresen kiolvassuk a ```w1_slave``` fájlból a hőmérsékletet. 

## Mit fogsz készíteni?

Egy DS18b20 szenzorból (és ha kell egy ellenállásból) álló áramkört készítünk, amiből kiolvasva a hőmérséklet értéket azt megjelenítjük a képernyőn néhány másodpercenként. 

## Mit tanulsz meg?

A hőmérő szenzor elkészítésével a következőket tanulod meg:

* Hogyan futtas terminál parancsokat Pythonból.
* Hogyan keress fájlokat és mappákat az operációs rendszeredben Pythonból a ```glob``` csomaggal.
* Hogyan olvassuk ki egy szöveges fájl tartalmát az ```open``` parancs használatával.
* Hogyan nyerjünk ki hasznos információt egy szöveges fájlból string parancsokat használva.

## A projekt részletekre bontása

* Elkészíteni az áramkört.
* Előkészíteni a kommunikációt a thermisztorral (```modprobe``` parancsok futtatása).
* Definiálni a hőmérsékletet tartalmazó fájl lokációját.
* Kiolvasni a fájl tartalmát az ```open``` parancs segítségével.
* Definiálni egy függvényt ami a fájlból kinyeri a hőmérsékletet.
* Másodpercenként megismételni a hőmérséklet olvasást.

## Áramköri elemek listája

a) [Raspberry PI](https://malnapc.hu/yis/raspberry-pi/rpi-panelek) 

b) DS18b20 szenzor: [itt vásárolhatsz](https://www.tme.eu/hu/details/ds18b20+/homerseklet-jelatalakitok/maxim-integrated/)

c) [Jumper wires female/male](https://www.ret.hu/shop/product/e-call/jumper-vezetek-szet_53-22-63) 

d) Ellenállás: [itt vásárolhatsz](https://www.tme.eu/hu/katalog/metal-film-tht-ellenallasok_112313/?s_order=desc&search=ellenallas&s_field=1000011)

## A kapcsolási rajz

<img src="prog00_schema.png" width=600 height=400 />

A fenti ábrához hasonlóan kapcsoljuk össze az áramköri elemeket és a Raspberry Pi-t. Az áramkör részletes összekötési magyarázata a [Motorok]() című bevezető leírásban található.

1) Az L293D driver földelés lábait kössük össze egymással és az átellenes oldallal (fekete drót). Egyben az áramforrás (4 db AAA elem a rajzon) negatív végét és a Raspberry Pi egyik földelését is kössük össze az L293D driver földelésével. Így minden elem azonos földelésen lesz.

2) Az áramforrás pozitív végét kapcsoljuk a driver *VS* lábára (*8*-as láb, piros drót). Ez adja a feszültséget a motor meghajtásához. 

3) Kapcsoljuk a Raspberry Pi +5V-os tüskéjét a driver *VSS* (*16*-os) lábával össze (fehér drót). Ez adja a driver működéséhez nélkülözhetetlen tápot. 

4) A Raspberry Pi *GPIO25*-ös tüskéjét kapcsoljuk az *EN1* (*1*-es) lábra (okkersárga drót). A driver ezen lába szolgál arra, hogy aktiválja a driver *IN1 (2), IN2 (7), OUT1 (3) és OUT2 (6)* lábait, amennyiben magas állapotba kerül. Alacsony állapotból az előbb említett lábak nem kapnak áramot. 

5) A Raspberry Pi *GPIO23*-as és *GPIO24*-es tüskéit kapcsoljuk az *IN1 (2), IN2 (7)* lábakra a driveren (szürke és narancssárga drótok). Felváltva aktiválva őket vagyunk képesek előre és hátrafele forgatni a motort attől függően, hogy épp melyik van magas állapotban. 

6) A motor két kimenetét (citromsárga és zöld drótok) kapcsoljuk a driver *OUT1 (3) és OUT2 (6)* lábaira. Az *IN1 és IN2* lábak állapota szabályozza, hogy ezek a lábak, azaz maga a motor, kap-e feszültséget.

**Kétszer is ellenőrizzük le, hogy a bekötésünk rendben van-e. A félrekötött tüskék nagyban növelik a motor vagy a Raspberry Pi tökremenési esélyeit. A motort SEMMIKÉPP SE tápláljuk és irányítsük direktben a Raspberry Pi-ről, mert az szinte biztosan a számítógép sérükéséhez vezet.**

## A kód
Nyissunk meg egy új python fájlt és mentsük el pl. ```dcmotor_test.py``` név alatt.

### A DC motor kipróbálása

Miután elkészítettük az áramkört, meg kell írnunk a kódot ami utasítja a Raspberry Pi-t, a motor mozgatására.

Először betöltjük a ```gpiozero``` modulból a ```Motor``` objektumot ami lehetővé teszik, hogy a Raspi kommunikáljon a motorral a hozzá tartozó vezérlővel a **GPIO** tüskéken keresztül. Emellett beimportáljuk a ```sleep``` függvényt is, hogy szünetet tarthassunk a python kód lefutásában. 

N.B. Az itt közölt parancsokat akár a terminálba is gépelhetjük, miután elindítottuk az ```ipython``` vagy ```python``` környezetet.

```dcmotor_test.py```:

In [ ]:
from gpiozero import Motor
from time import sleep

Az objektum beolvasása után inicializálunk egy ```Motor``` objektumot, amit ```motor```-nak nevezünk el. Egyben megmondjuk az objektumnak, hogy a fizikai motor előre forgatásért felelő drótját a *23*-as számú **GPIO** tüskére csatlakoztattuk, míg a hátrafelé forgatásért felelő drótot a *24*-es számú **GPIO** tüskére. Emellett a driveren a kimeneteket aktiváló drótot a *25*-ös számú **GPIO** tüskére csatlakoztattuk. 

```dcmotor_test.py```:

In [ ]:
from gpiozero import Motor
from time import sleep

motor = Motor(forward=23, backward = 24, enable=25)

Ezután készen állunk arra, hogy leteszteljük a motor mozgatását és az objektumhoz tartozó egyéb metódusok működését. Elsőnek a motort előre forgatjuk ```motor.forward()```, 5 másodpercen keresztül. Majd lecsökkentjük a forgás sebességét, ```motor.forward(speed=0.8)```. A ```speed``` értéke 0 és 1 között változhat, de ha túl kicsire vesszük a sebességet, nem biztos, hogy a motor rendelkezni fog elég nyomatékkal, hogy forogjon, ami ha tartós ideig áll fenn, akkor leéghet a motor. Ezután leellenőrizzük, hogy a motor valóban forog-e még a ```motor.is_active``` metódussal. A forgási sebességet meghatározó értéket a ```motor.value``` metódussal kaphatjuk vissza, amit ki is nyomtatunk a képernyőre. Majd hadjuk forogni a motort 5 másodpercig, és megállítjuk a motort, ```motor.stop()```.

```dcmotor_test.py```:

In [ ]:
from gpiozero import Motor
from time import sleep

motor = Motor(forward=23, backward = 24, enable=25)

motor.forward()
sleep(5)

motor.forward(speed=0.8)
print(motor.is_active)
print(motor.value)
sleep(5)
motor.stop()

A motorhoz tartozó további metódusok gyakorlássaképp próbáljuk ki hátrafele forgatni a motor kerekét a ```motor.backward()``` paranccsal, ahol szintén megadhatjuk a forgási sebességet a ```speed``` argumentum segítségével. Ezután kinyomtathatjuk a sebesség értékét, hogy lássuk helyesen lett beállítva és hadjuk a motort forogni 5 másodpercen keresztül. A ```motor.reverse()``` paranccsal a forgási irányt tudjuk változtatni, miközben a beállított sebesség nem változik. Hagyjuk ismét a motort forogni 5 másodpercig. Ezután megállítjuk a motort a ```motor.stop()``` paranccsal és a ```motor.is_active``` paranccsal leellenőrizzük, hogy valóban leállt-t. Ha az értéke ```False```, akkor sikeresen megállítottuk a motort. 

```dcmotor_test.py```:

In [ ]:
from gpiozero import Motor
from time import sleep

motor = Motor(forward=23, backward = 24, enable=25)

motor.forward()
sleep(5)

motor.forward(speed=0.8)
print(motor.is_active)
print(motor.value)
sleep(5)
motor.stop()

motor.backward(speed=0.9)
print(motor.value)
sleep(5)
motor.reverse()
sleep(5)
motor.stop()
print(motor.is_active)

## A projekt tesztelése

Miután összeszereltük az áramkört és a kódot is megírtuk, amit pl. ```dcmotor_test.py``` név alatt mentettünk el, megnyithatunk a Raspberry Pi operációs rendszerén egy terminált. A terminálban a ```cd 'mappa név'``` paranccsal elnavigálunk abba a mappába, ahova a ```dcmotor_test.py```-t elmentettük. Ott begépelve a ```python dcmotor_test.py``` parancsot, letesztelhetjük a programunk működését. Ha minden jól megy akkor a gomb lenyomására a motor elkezd forogni, majd megváltozik a sebessége, majd megáll. Utána ismét forogni kezd, csak az ellenkező irányba, majd ismét irányt vált és végül megáll.

Hibaüzenetek esetén ki kell deríteni mi lehetett a probléma, pl. elgépelés, egy modul hiányzik, sorok megfelelő behúzása, idézőjel lemaradása stb. A hibaüzenet legtöbbször segít abban, hogy melyik sorban találta a hibát és hogy mi volt az. Egy kis gyakorlással bele lehet jönni azok értelmezésébe, valamint interneten is rá lehet keresni a hibaüzenet jelentésére és annak lehetséges elhárítására.

## Mit lehet javítani/továbbfejleszteni?

* Teszteljük le, mekkora sebességgel képes forogni a motor.
* Változtassuk meg az időt, amíg a motor az egyik, majd a másik irányba forog.

Írd meg kommentben, hogy szerinted mivel lehetne még feldobni ezt a kis programot!

## Referencia

1) gpiozero leírás - [https://gpiozero.readthedocs.io/en/stable/recipes.html](https://gpiozero.readthedocs.io/en/stable/recipes.html)

2) Motor objektum leírása - [https://gpiozero.readthedocs.io/en/stable/api_output.html#motor](https://gpiozero.readthedocs.io/en/stable/api_output.html#motor)

3) L293D driver leírása - [https://www.ti.com/lit/ds/symlink/l293.pdf](https://www.ti.com/lit/ds/symlink/l293.pdf)